In [126]:
import pandas as pd

# Load the datasets into notebook
train_data = pd.read_csv('train.csv')
x_test = pd.read_csv('x_test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

# Display first few rows of each dataset for initial analysis
train_data.head(), x_test.head(), sample_submission.head()

(   id     Col 1     Col 2     Col 3     Col 4     Col 5     Col 6     Col 7  \
 0   0  0.038076  0.050680  0.061696  0.021872 -0.044223 -0.034821 -0.043401   
 1   1 -0.001882 -0.044642 -0.051474 -0.026328 -0.008449 -0.019163  0.074412   
 2   2  0.085299  0.050680  0.044451 -0.005671 -0.045599 -0.034194 -0.032356   
 3   3 -0.089063 -0.044642 -0.011595 -0.036656  0.012191  0.024991 -0.036038   
 4   4  0.005383 -0.044642 -0.036385  0.021872  0.003935  0.015596  0.008142   
 
       Col 8     Col 9  ...    Col 56    Col 57    Col 58    Col 59    Col 60  \
 0 -0.002592  0.019908  ... -0.022038 -0.031125 -0.000922  0.033494  0.000852   
 1 -0.039493 -0.068330  ... -0.011564  0.012973  0.023783 -0.023815 -0.094506   
 2 -0.002592  0.002864  ... -0.022063 -0.018016  0.004913  0.032956  0.018281   
 3  0.034309  0.022692  ... -0.009984 -0.003373 -0.019109  0.008159  0.001898   
 4 -0.002592 -0.031991  ... -0.024091 -0.026846 -0.029687  0.030984  0.014489   
 
      Col 61    Col 62    Col 

In [127]:
from sklearn.preprocessing import StandardScaler

# Extracting features and target from the training data
X_train = train_data.drop(columns=['id', 'y'])
y_train = train_data['y']

# Initializing the scaler
scaler = StandardScaler()

# Fitting the scaler on the training data and transforming both training and test data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(x_test.drop(columns=['id']))

X_train_scaled[:5], X_test_scaled[:5]  # Display first 5 rows of scaled data for both training and test sets

(array([[ 9.79154374e-01,  1.11960758e+00,  1.36860714e+00,
          5.74283896e-01, -8.89928871e-01, -6.64196046e-01,
         -9.51007755e-01,  7.10181293e-02,  5.95400472e-01,
         -2.62269997e-01, -3.51264292e-01,  6.10648506e-01,
         -5.95255126e-01,  2.60783492e-02, -2.22070784e-01,
         -1.31394451e-01, -6.11542880e-01, -5.86563565e-01,
         -5.78683361e-01,  5.24103969e-01,  7.70315361e-01,
          2.71409789e-02, -1.03778190e+00, -8.07639839e-01,
         -7.43266530e-01, -2.62183891e-01,  1.13441831e-02,
         -5.93170085e-01,  1.32067282e+00,  2.64185903e-01,
         -1.29541595e+00, -1.15701137e+00, -6.08538929e-01,
         -5.54978988e-01,  2.53310314e-01, -7.30854859e-01,
          2.05667374e-01, -1.58061502e+00, -1.31991750e+00,
         -8.73702554e-01, -5.06521776e-01,  1.44609676e-01,
         -8.30228631e-01, -8.20272280e-01, -6.22852725e-01,
         -3.03465326e-01, -3.03015277e-01, -2.34186725e-01,
         -5.13534270e-01, -6.63078504e-0

In [128]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Initialize the linear regression model
linear_regression_model = LinearRegression()

# Train the model
linear_regression_model.fit(X_train_scaled, y_train)

# Predict on training data for evaluation
y_train_pred = linear_regression_model.predict(X_train_scaled)

# Calculate the MSE on training data
mse_train = mean_squared_error(y_train, y_train_pred)

mse_train

1710.6670090547075

In [129]:
from sklearn.linear_model import Ridge, Lasso

# Initialize Ridge and Lasso models
ridge_model = Ridge()
lasso_model = Lasso()

# Train the models
ridge_model.fit(X_train_scaled, y_train)
lasso_model.fit(X_train_scaled, y_train)

# Predict on training data for evaluation
y_train_pred_ridge = ridge_model.predict(X_train_scaled)
y_train_pred_lasso = lasso_model.predict(X_train_scaled)

# Calculate the MSE on training data for both models
mse_train_ridge = mean_squared_error(y_train, y_train_pred_ridge)
mse_train_lasso = mean_squared_error(y_train, y_train_pred_lasso)

mse_train_ridge, mse_train_lasso


(1813.7723061914548, 2031.209448053145)

In [130]:
from sklearn.model_selection import GridSearchCV

# Define the hyperparameters to search
param_grid = {
    'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
}

# Grid search with cross-validation for Ridge regression
ridge_search = GridSearchCV(Ridge(), param_grid, cv=5, scoring='neg_mean_squared_error')
ridge_search.fit(X_train_scaled, y_train)

# Grid search with cross-validation for Lasso regression
lasso_search = GridSearchCV(Lasso(), param_grid, cv=5, scoring='neg_mean_squared_error')
lasso_search.fit(X_train_scaled, y_train)

# Get the best parameters and scores for both models
best_alpha_ridge = ridge_search.best_params_['alpha']
best_mse_ridge = -ridge_search.best_score_

best_alpha_lasso = lasso_search.best_params_['alpha']
best_mse_lasso = -lasso_search.best_score_

best_alpha_ridge, best_mse_ridge, best_alpha_lasso, best_mse_lasso



/Users/mihir/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.484e+04, tolerance: 7.976e+01
  model = cd_fast.enet_coordinate_descent(
/Users/mihir/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.813e+04, tolerance: 7.245e+01
  model = cd_fast.enet_coordinate_descent(
/Users/mihir/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duali

(100.0, 3692.4353151650657, 10.0, 3671.1123626328745)

In [131]:
# Predict on the test data using the best Lasso model
y_test_pred = lasso_search.best_estimator_.predict(X_test_scaled)

# Preparing the submission dataframe
submission = x_test[['id']].copy()
submission['y'] = y_test_pred

submission.head()

,id,y
0,160,107.887927
1,161,227.246385
2,162,138.843740
3,163,176.657423
4,164,149.088094


In [132]:
from sklearn.ensemble import GradientBoostingRegressor

# Initialize the gradient boosting regressor
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)

# Train the model
gb_model.fit(X_train_scaled, y_train)

# Predict on training data for evaluation
y_train_pred_gb = gb_model.predict(X_train_scaled)

# Calculate the MSE on training data
mse_train_gb = mean_squared_error(y_train, y_train_pred_gb)

mse_train_gb


71.12658100462362

In [133]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation on the Gradient Boosting Regressor
cv_scores = cross_val_score(gb_model, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')

# Calculate mean and standard deviation of cross-validation scores
mean_mse_gb_cv = -cv_scores.mean()
std_mse_gb_cv = cv_scores.std()

mean_mse_gb_cv, std_mse_gb_cv


(4442.096981260921, 510.3552826913761)

In [134]:
# Define hyperparameters for grid search
param_grid_gb = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.05, 0.1, 0.5],
    'max_depth': [3, 4, 5, 6]
}

# Grid search with cross-validation for Gradient Boosting Regressor
gb_search = GridSearchCV(GradientBoostingRegressor(random_state=42), param_grid_gb, cv=5, scoring='neg_mean_squared_error')
gb_search.fit(X_train_scaled, y_train)

# Get the best parameters and score
best_params_gb = gb_search.best_params_
best_mse_gb = -gb_search.best_score_

print(best_params_gb, best_mse_gb)

# Predict on the test data using the best Gradient Boosting Regressor from randomized search
y_test_pred_gb = gb_search.best_estimator_.predict(X_test_scaled) #test based on final x_test values

{'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 200} 4120.996225188528


In [135]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=42)

# For Linear Regression
linear_regression_model.fit(X_train, y_train)
y_val_pred = linear_regression_model.predict(X_val)
mse_val_lr = mean_squared_error(y_val, y_val_pred)

# For Ridge Regression with best alpha
ridge_model_best = Ridge(alpha=best_alpha_ridge)
ridge_model_best.fit(X_train, y_train)
y_val_pred_ridge = ridge_model_best.predict(X_val)
mse_val_ridge = mean_squared_error(y_val, y_val_pred_ridge)

# For Lasso Regression with best alpha
lasso_model_best = Lasso(alpha=best_alpha_lasso)
lasso_model_best.fit(X_train, y_train)
y_val_pred_lasso = lasso_model_best.predict(X_val)
mse_val_lasso = mean_squared_error(y_val, y_val_pred_lasso)

# For Gradient Boosting with best parameters
gb_model_best = GradientBoostingRegressor(**best_params_gb, random_state=42)
gb_model_best.fit(X_train, y_train)
y_val_pred_gb = gb_model_best.predict(X_val)
mse_val_gb = mean_squared_error(y_val, y_val_pred_gb)

print("Linear Regression MSE:", mse_val_lr)
print("Ridge Regression MSE:", mse_val_ridge)
print("Lasso Regression MSE:", mse_val_lasso)
print("Gradient Boosting MSE:", mse_val_gb)

Linear Regression MSE: 11737.49025311329
Ridge Regression MSE: 4016.9671706988183
Lasso Regression MSE: 4180.076984649635
Gradient Boosting MSE: 3445.2228348530252


In [136]:
# Update the submission dataframe with new predictions
submission['y'] = y_test_pred_gb

submission.head()

# Save the submission dataframe to a CSV file
submission_file_path = "submissions_test.csv"
submission.to_csv(submission_file_path, index=False)

submission_file_path

'submissions_test.csv'